In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

import os 
from glob import glob 

from PIL import Image, ImageOps
import albumentations as A
import cv2

from tqdm import tqdm 

In [2]:
df = pd.read_csv('../data/train.csv')

In [4]:
ids = []
targets = []
data_path = '../data/train/'
for index, ID, target in tqdm(df.itertuples(), desc='Image rotation', mininterval=0.1):
    image_path = os.path.join(data_path, ID)

    # 90도 회전 
    id_90 = 'r90_'+ ID
    ids.append(id_90)
    targets.append(target)
    rotate90_image = np.array(ImageOps.exif_transpose(Image.open(image_path).rotate(90, expand=True)))
    Image.fromarray(rotate90_image).save(os.path.join(data_path, id_90))
    
    # 180도 회전 
    id_180 = 'r180_'+ ID
    ids.append(id_180)
    targets.append(target)
    rotate180_image = np.array(ImageOps.exif_transpose(Image.open(image_path).rotate(180, expand=True)))
    Image.fromarray(rotate180_image).save(os.path.join(data_path, id_180))

    # 270도 회전 
    id_270 = 'r270_'+ ID
    ids.append(id_270)
    targets.append(target)
    rotate270_image = np.array(ImageOps.exif_transpose(Image.open(image_path).rotate(270, expand=True)))
    Image.fromarray(rotate270_image).save(os.path.join(data_path, id_270))

rotate_data = {
    'ID' : ids,
    'target' : targets
}
rotate_df = pd.DataFrame(rotate_data)    
df = pd.concat([df, rotate_df])

Image rotation: 1570it [00:39, 39.73it/s]


In [6]:
H_flip = A.HorizontalFlip(always_apply=True, p=1)
V_flip = A.VerticalFlip(always_apply=True, p=1)

In [7]:
ids = []
targets = []
data_path = '../data/train/'
for index, ID, target in tqdm(df.itertuples(), desc='Image flip', mininterval=0.1):
    image_path = os.path.join(data_path, ID)
    image = np.array(Image.open(image_path))

    if ID.startswith('r90') or ID.startswith('r270'):
        # V flip
        id_vflip = 'Vflip_'+ ID
        ids.append(id_vflip)
        targets.append(target)
        vflip_image = V_flip(image=image)['image']
        Image.fromarray(vflip_image).save(os.path.join(data_path, id_vflip))
    else:
        # H flip
        id_hflip = 'Hflip_'+ ID
        ids.append(id_hflip)
        targets.append(target)
        hflip_image = H_flip(image=image)['image']
        Image.fromarray(hflip_image).save(os.path.join(data_path, id_hflip))
    
flip_data = {
    'ID' : ids,
    'target' : targets
}
flip_df = pd.DataFrame(flip_data)    
df = pd.concat([df, flip_df])

Image flip: 6280it [00:46, 134.02it/s]


In [9]:
transforms = A.Compose([
    A.OneOf([
        A.GridDistortion(num_steps=5, distort_limit=0.3, interpolation=1, border_mode=4, value=None, mask_value=None, always_apply=True, p=1),
        A.ElasticTransform(always_apply=True, p=1, alpha=1.0, sigma=50.0, alpha_affine=50.0, interpolation=0, border_mode=1, value=(0, 0, 0), mask_value=None, approximate=False),
        A.OpticalDistortion(always_apply=True, p=1, distort_limit=(-0.3, -0.1)),
        A.OpticalDistortion(always_apply=True, p=1, distort_limit=(0.1, 0.3)),
    ], p=0.85),
    A.SomeOf([
        A.RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=1),
        A.HueSaturationValue(hue_shift_limit=15, sat_shift_limit=25, val_shift_limit=20, p=1),
        A.MultiplicativeNoise(p=1, multiplier=(1, 1.5), per_channel=True),
        A.Equalize(p=1, mode='cv', by_channels=True),
    ], n=2, p=0.85),
    A.OneOf([
        A.Rotate(limit=(10, 30), border_mode=cv2.BORDER_CONSTANT, p=1),
        A.Rotate(limit=(150, 170), border_mode=cv2.BORDER_CONSTANT, p=1),
        A.Rotate(limit=(190, 210), border_mode=cv2.BORDER_CONSTANT, p=1),
        A.Rotate(limit=(330, 350), border_mode=cv2.BORDER_CONSTANT, p=1),
    ], p=1),
    A.CoarseDropout(p=0.5, max_holes=40, max_height=15, max_width=15, min_holes=8, min_height=8, min_width=8),
    A.Equalize(p=0.5, mode='cv', by_channels=True),
    A.OneOf([
        A.Blur(blur_limit=(3, 4), p=1),
        A.MotionBlur(blur_limit=(3, 5), p=1),
        A.Downscale(scale_min=0.455, scale_max=0.5, interpolation=2, p=1),
    ], p=0.5),
    A.GaussNoise(var_limit=(100, 800), per_channel=True, p=0.5),
])

In [11]:
ids = []
targets = []
data_path = '../data/train/'
for index, ID, target in tqdm(df.itertuples(), desc='Image augmentation', mininterval=0.1):
    image_path = os.path.join(data_path, ID)
    image = np.array(Image.open(image_path))
    if target == 13:
        n = 13
    elif target == 14:
        n = 20
    elif target == 1:
        n = 21
    else:
        n = 10
    for i in range(n):
        transformed_image = transforms(image=image)['image']
        image_ID = f'tf{i}_' + ID 
        ids.append(image_ID)
        targets.append(target)
        Image.fromarray(transformed_image).save(os.path.join(data_path, image_ID))
    
aug_data = {
    'ID' : ids,
    'target' : targets
}
aug_df = pd.DataFrame(aug_data)    
df = pd.concat([df, aug_df])

Image augmentation: 12560it [2:07:36,  1.64it/s]


In [15]:
df.to_csv('../data/add_transformed_train.csv', index=False)